# Prediction d'un devis sur les messages IT

## 1. Création du dataset

In [1]:
import sys
sys.executable

'/usr/local/Caskroom/miniconda/base/envs/datascience/bin/python'

In [3]:
import pandas
df_dirty = pandas.read_csv("proposal_messages_IT.csv")
df_dirty.head()

,request_id,message_id,message_send_date,body,list_attachments_type,Contains a proposal ?
0,1881838,24069071,2019-01-08 21:56:54.0,Hola Laura!\n\nGrazie per la tua richiesta di ...,NaN,False
1,1881838,25448168,2019-02-28 19:14:05.0,Ciao Laura!\n\nCome stai?\nScusa se ti scrivo ...,NaN,False
2,1881838,25451448,2019-02-28 21:55:02.0,"Va bene Laura, \ngrazie mille! Allora domani t...",NaN,False
3,1881838,25468771,2019-03-01 15:02:23.0,"Buongiorno Laura,\n\ncome stai?\nTi mando in a...",application/pdf attached,True
4,1881838,25596417,2019-03-06 16:02:33.0,Ciao Laura! \n\nCome stai?\nMi fa molto piacer...,NaN,False


In [28]:
df_dirty['contain_proposal'] = df_dirty['Contains a proposal ?'].apply(lambda x: 1 if x is True else 0)
df = df_dirty[['body','contain_proposal']].copy()
df.head()

,body,contain_proposal
0,Hola Laura!\n\nGrazie per la tua richiesta di ...,0
1,Ciao Laura!\n\nCome stai?\nScusa se ti scrivo ...,0
2,"Va bene Laura, \ngrazie mille! Allora domani t...",0
3,"Buongiorno Laura,\n\ncome stai?\nTi mando in a...",1
4,Ciao Laura! \n\nCome stai?\nMi fa molto piacer...,0


## 2. Cleaning et préparation du texte

### 2.1 minuscule, ponctuation, tokenisation

In [29]:
import nltk
import re
import string
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

table = str.maketrans('', '', string.punctuation)

print(df.loc[0]['body'])
df['body_1'] = df['body'].apply(lambda text: str(text).lower())
df['body_1'] = df['body_1'].str.replace("\r", " ")
df['body_1'] = df['body_1'].str.replace("\n", " ")
df['body_1'] = df['body_1'].apply(lambda text: re.split(r'\W+', text)) 
df['body_1'] = df['body_1'].apply(lambda words: [w.translate(table) for w in words]) 
print("#"*100)
print(df.loc[0]['body_1'])

Hola Laura!

Grazie per la tua richiesta di viaggio sul sito di Evaneos.it. 
Mi chiamo Filomena, sono italiana e vivo in Argentina da diversi anni. La mia passione per questo paese mi ha portata a lavorare con l' agenzia Argentina On The Go, specializzata nella creazione di viaggi su misura in Argentina e Cile. 

Sara' davvero un piacere per me farvi scoprire i luoghi piu' belli dell'Argentina, ma prima d'inviarti una proposta adatta alle vostre esigenze, potresti per favore dettagliare i vari punti indicati di seguito?


- Avete già prenotato i voli internazionali? Con questa informazione sarò in grado di adattare una proposta anche per i voli interni, 
- Parlate Inglese o Spagnolo? Per la maggior parte delle escursioni le guide sono in Spagnolo-Inglese, e il supplemento per la guida in Italiano può essere costoso, quindi capire un po di qualsiasi di queste lingue potrebbe essere vantaggioso, 


Quando avrò ricevuto queste informazioni , ti invierò una prima proposta di viaggio su mis

### 2.2 stop_words


In [30]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('italian')]
    return words
df['body_2'] = df['body_1'].apply(lambda x: remove_stopwords(x))
print(df.loc[0]['body_2'])

['hola', 'laura', 'grazie', 'richiesta', 'viaggio', 'sito', 'evaneos', 'it', 'chiamo', 'filomena', 'italiana', 'vivo', 'argentina', 'diversi', 'anni', 'passione', 'paese', 'portata', 'lavorare', 'agenzia', 'argentina', 'on', 'the', 'go', 'specializzata', 'creazione', 'viaggi', 'misura', 'argentina', 'cile', 'sara', 'davvero', 'piacere', 'me', 'farvi', 'scoprire', 'luoghi', 'piu', 'belli', 'argentina', 'prima', 'd', 'inviarti', 'proposta', 'adatta', 'esigenze', 'potresti', 'favore', 'dettagliare', 'vari', 'punti', 'indicati', 'seguito', 'già', 'prenotato', 'voli', 'internazionali', 'informazione', 'grado', 'adattare', 'proposta', 'voli', 'interni', 'parlate', 'inglese', 'spagnolo', 'maggior', 'parte', 'escursioni', 'guide', 'spagnolo', 'inglese', 'supplemento', 'guida', 'italiano', 'può', 'essere', 'costoso', 'quindi', 'capire', 'po', 'qualsiasi', 'lingue', 'potrebbe', 'essere', 'vantaggioso', 'quando', 'ricevuto', 'informazioni', 'invierò', 'prima', 'proposta', 'viaggio', 'misura', 'ri

### 2.3 lemmatization

In [38]:
nlp = spacy.load('it_core_news_sm')

def lemmatize_text(text):
    doc = nlp(" ".join(text))
    return [word.lemma_ for word in doc]

df['body_3'] = df['body_2'].apply(lambda x: lemmatize_text(x))
df['body_cleaned'] = df['body_3'].apply(lambda x: " ".join(x))
print(df.loc[0]['body_3'])

['hola', 'laura', 'grazia', 'richiedere', 'viaggiare', 'sito', 'evaneos', 'it', 'chiamare', 'filomena', 'italiano', 'vivere', 'argentino', 'diverso', 'anno', 'passione', 'paese', 'portato', 'lavorare', 'agenzia', 'argentino', 'on', 'the', 'go', 'specializzato', 'creazione', 'viaggo', 'misurare', 'argentino', 'cile', 'sara', 'davvero', 'piacere', 'me', 'farvi', 'scoprire', 'luogo', 'piu', 'bello', 'argentino', 'primo', 'd', 'inviarti', 'proporre', 'adatto', 'esigenza', 'potere', 'favore', 'dettagliare', 'varo', 'punto', 'indicato', 'seguitare', 'già', 'prenotare', 'volo', 'internazionale', 'informazione', 'grado', 'adattare', 'proporre', 'volo', 'interno', 'parlato', 'inglese', 'spagnolo', 'maggior', 'partire', 'escursione', 'guida', 'spagnolo', 'inglese', 'supplementare', 'guidare', 'italiano', 'potere', 'essere', 'costoso', 'quindi', 'capire', 'po', 'qualsiasi', 'lingua', 'potere', 'essere', 'vantaggioso', 'quando', 'ricevere', 'informazione', 'inviare', 'primo', 'proporre', 'viaggiar

## 3. Vectorisation et TF_IDF

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(df['body_cleaned'])
print(vectorizer.vocabulary_)

{'hola': 5116, 'laura': 5799, 'grazia': 4990, 'richiedere': 7898, 'viaggiare': 9605, 'sito': 8541, 'evaneos': 4364, 'it': 5584, 'chiamare': 3001, 'filomena': 4589, 'italiano': 5592, 'vivere': 9682, 'argentino': 2080, 'diverso': 3892, 'anno': 1950, 'passione': 6984, 'paese': 6867, 'portato': 7260, 'lavorare': 5800, 'agenzia': 1732, 'on': 6720, 'the': 9097, 'go': 4934, 'specializzato': 8696, 'creazione': 3527, 'viaggo': 9609, 'misurare': 6330, 'cile': 3070, 'sara': 8250, 'davvero': 3655, 'piacere': 7146, 'me': 6200, 'farvi': 4473, 'scoprire': 8329, 'luogo': 5956, 'piu': 7198, 'bello': 2498, 'primo': 7443, 'inviarti': 5538, 'proporre': 7516, 'adatto': 1612, 'esigenza': 4281, 'potere': 7290, 'favore': 4494, 'dettagliare': 3762, 'varo': 9539, 'punto': 7577, 'indicato': 5338, 'seguitare': 8395, 'già': 4912, 'prenotare': 7374, 'volo': 9696, 'internazionale': 5488, 'informazione': 5389, 'grado': 4969, 'adattare': 1608, 'interno': 5491, 'parlato': 6941, 'inglese': 5396, 'spagnolo': 8680, 'maggi

In [40]:
X = vectorizer.transform(df['body_cleaned'])
X.shape

(1598, 9855)

In [ ]:
X.toarray()

In [ ]:
Y = df['contain_proposal']

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X)
X_tfidf.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, 
                                                    Y)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
print(X_test.shape)
print(y_test.shape)
print(y_test)

In [ ]:
from sklearn import naive_bayes
from sklearn.metrics import accuracy_score

# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train,y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(X_test)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, y_test)*100)
print(y_test)

In [ ]:
result = pandas.DataFrame()
result['X'] = vectorizer.inverse_transform(X_test)
result['Y'] = y_test
result['Y_predicted'] = predictions_NB
result.head()

In [ ]:
corrM = result.corr() 
  
corrM 

In [ ]:
predictions_NB

In [ ]:
y_train

In [ ]:
vectorizer.inverse_transform(X_test)

In [ ]:
from sklearn.linear_model import SGDClassifier
text_clf = SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                           max_iter=5, tol=None)

text_clf.fit(X_train,y_train)

predicted = text_clf.predict(X_test)
print("SVN -> ",accuracy_score(predicted, y_test)*100)


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predicted)

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(confusion_matrix(y_test, predicted),annot = True)